In [ ]:
# Import required libraries
import geopandas as gpd
import numpy as np
import os
import pandas as pd

# Define a list of shapefile paths (user should provide these paths)
shapefile_paths = [
    # Example: 'path/to/shapefile1.shp',
    # Add your shapefile paths here...
]

# Define function for calculating total damage
def calculate_damage(road_shapefile, flood_shapefile):
    # Read the road and flood shapefiles
    Road = gpd.read_file(road_shapefile)
    Flood = gpd.read_file(flood_shapefile)

    # Perform intersection between road and flood shapefiles
    flood_road_intersection = gpd.overlay(Road, Flood, how='intersection')

    # Add columns for damage calculations
    flood_road_intersection['damage_factor'] = np.nan
    flood_road_intersection['unit_damage'] = np.nan
    flood_road_intersection['area'] = np.nan
    flood_road_intersection['Total_damage'] = np.nan

    # Define conditions based on 'Maximum(m)' column
    conditions = [
        (flood_road_intersection['Maximum(m)'] == 0),
        (flood_road_intersection['Maximum(m)'] > 0) & (flood_road_intersection['Maximum(m)'] <= 0.5),
        (flood_road_intersection['Maximum(m)'] > 0.5) & (flood_road_intersection['Maximum(m)'] <= 1),
        (flood_road_intersection['Maximum(m)'] > 1) & (flood_road_intersection['Maximum(m)'] <= 1.5),
        (flood_road_intersection['Maximum(m)'] > 1.5) & (flood_road_intersection['Maximum(m)'] <= 2),
        (flood_road_intersection['Maximum(m)'] > 2) & (flood_road_intersection['Maximum(m)'] <= 3),
        (flood_road_intersection['Maximum(m)'] > 3) & (flood_road_intersection['Maximum(m)'] <= 4),
        (flood_road_intersection['Maximum(m)'] > 4) & (flood_road_intersection['Maximum(m)'] <= 5),
        (flood_road_intersection['Maximum(m)'] > 5)
    ]

    # Define corresponding damage factors
    damage_factors = [0, 0.21, 0.37, 0.60, 0.71, 0.81, 0.89, 0.97, 1.0]

    # Apply conditions to assign the damage factor
    flood_road_intersection['damage_factor'] = np.select(conditions, damage_factors, default=np.nan)

    # calculate the'unit_damage'
    flood_road_intersection['unit_damage'] = flood_road_intersection['damage_factor'] * 2.96

    # Calculate the area of each geometry
    flood_road_intersection['area'] = flood_road_intersection.geometry.area

    # Calculate total damage
    flood_road_intersection['total_damage'] = flood_road_intersection['area'] * flood_road_intersection['unit_damage']

    # Calculate the sum of total damage
    total_damage_sum = flood_road_intersection['total_damage'].sum()

    # Convert total damage to INR using currency factors for 2023
    Damage_INR = total_damage_sum / 0.0165
    Damage_INR_2023 = Damage_INR * 216.86 / 100
    Damage_INR_2023_cr = Damage_INR_2023 / (10**7)

    return Damage_INR_2023_cr

# Load the road shapefile once (user should provide the path)
road_shapefile = 'path/to/road_shapefile.shp'  # Modify with the actual path

# List to store the damage results
damage_results = []

# Loop through all flood shapefiles and calculate total damage
for shapefile_path in shapefile_paths:
    damage = calculate_damage(road_shapefile, shapefile_path)
    damage_results.append({
        "shapefile": os.path.basename(shapefile_path),
        "total_damage_in_cr": damage
    })
    print(f"Total Damage for {shapefile_path}: {damage} crore INR")

# Convert the results to a pandas DataFrame
damage_df = pd.DataFrame(damage_results)

# Display the DataFrame with all results
print(damage_df)
